### Задание buildings1

In [1]:
from random import randint

Инициализация библиотеки bokeh для визуализации (для установки `pip install bokeh`,  
либо `conda install bokeh`, либо через Пуск/Anaconda Navigator):

In [2]:
from bokeh.io import output_notebook, push_notebook
import bokeh.plotting
output_notebook()

Loading BokehJS ...

**A.** Реализуйте методы класса `Building1` так, чтобы выражение `A & B` определяло, пересекаются  
 ли здания A и B путём такой проверки:  
&nbsp; &nbsp;   – попадает ли хотя бы один из четырёх углов здания A в «габариты» здания B и наоборот,  
&nbsp; &nbsp;   – попадает ли хотя бы один из углов B в габариты A.

In [3]:
class Building1:
    """Реализация класса здание с определением пересечений способом 1"""
    # ^^^ по таким строкам автоматически генерируется документация, попробуйте
    # (после выполнения этой ячейки) написать help(Building1) или нажать shift-tab на
    # слове Building1. Это называется docstrings. Как обычно, отступ слева
    # должен быть такой же, как у вложенного блока (в данном случае, как у методов класса).
    
    def __init__(self, x1, y1, x2, y2):
        """Создаёт здание по координатам двух противоположных его углов"""
        self.x1, self.x2 = (x1, x2) if x1 < x2 else (x2, x1)
        self.y1, self.y2 = (y1, y2) if y1 < y2 else (y2, y1)

    def __repr__(self):
        return f'Building1({self.x1}, {self.y1}, {self.x2}, {self.y2})'

    def __contains__(self, point):
        """Определяет, находится ли точка point внутри здания.
        Вызывается при исполнении выражения '(x, y) in building' """
        x, y = point
        return self.x1 <= x <= self.x2 and self.y1 <= y <= self.y2

    def corners(self):
        """Возвращает список из координат всех четырёх углов здания"""
        return [(self.x1, self.y1), (self.x2, self.y2), (self.x1, self.y2), (self.x2, self.y1)]

    def __and__(self, other):
        """Реализует оператор &, который определяет, пересекаются ли 
        здания self и other первым способом (через углы)"""
        # для вычисления b1 & b2 питон вызывает метод b1.__and__(b2) 
        # здесь удобно использовать метод corners и перегруженный оператор in
        b1, b2 = self, other    # удобно переобозвать их - так короче и симметричнее
        if any([с in b2 for с in b1.corners()]):
            return True
        return any([с in b1 for с in b2.corners()])
    

In [4]:
def test(got, expected):
    print(' OK ' if got == expected else '  X ',
          f'Получено: {got} | Ожидалось: {expected}')

In [5]:
test(len(Building1(0, 1, 2, 3).corners()), 4)
test((0, 3) in Building1(0, 1, 2, 3).corners(), True)

 OK  Получено: 4 | Ожидалось: 4
 OK  Получено: True | Ожидалось: True


In [6]:
b = Building1(0, 0, 2, 4)
test((0, 0) in b, True)
test((1, 3) in b, True)
test((3, 1) not in b, True)

 OK  Получено: True | Ожидалось: True
 OK  Получено: True | Ожидалось: True
 OK  Получено: True | Ожидалось: True


In [7]:
test(Building1(0, 0, 1, 1) & Building1(2, 2, 3, 3), False)
test(Building1(0, 0, 2, 2) & Building1(1, 1, 3, 3), True)
test(Building1(2, 2, 0, 0) & Building1(3, 3, 1, 1), True)

 OK  Получено: False | Ожидалось: False
 OK  Получено: True | Ожидалось: True
 OK  Получено: True | Ожидалось: True


In [8]:
def vis(b1, b2):      # визуализация пары зданий
    p = bokeh.plotting.figure(plot_width=750, plot_height=300)
    blue1, blue2 = "#dae8fc", '#6c8ebf'
    green1, green2 = "#d5e8d4", '#82b366'
    r1 = p.quad(top=[b1.y2], bottom=[b1.y1], left=[b1.x1], right=[b1.x2], 
           fill_color=blue1, line_color=blue2, fill_alpha=0.5)
    r2 = p.quad(top=[b2.y2], bottom=[b2.y1], left=[b2.x1], right=[b2.x2], 
           fill_color=green1, line_color=green2, fill_alpha=0.5)
    bokeh.plotting.show(p, notebook_handle=True)
    return r1, r2

In [9]:
b1, b2 = Building1(0, 0, 4, 3), Building1(2, 2, 6, 5)
vis(b1, b2)
test(b1 & b2, True)

 OK  Получено: True | Ожидалось: True


In [10]:
b1, b2 = Building1(0, 0, 10, 5), Building1(2, 1, 8, 4)
vis(b1, b2);
test(b1 & b2, True)
test(b2 & b1, True)

 OK  Получено: True | Ожидалось: True
 OK  Получено: True | Ожидалось: True


**B.** Реализуйте класс Building2, наследующийся от Building1, в котором пересечение  
определялось бы правильно.

В референсном решении четыре оператора сравнения. Если у вас больше, вероятно, что-то вы   
делаете неоптимально.

In [11]:
class Building2(Building1):
    """Реализация класса здание с определением пересечений 
    вторым способом (через проекции)"""
    
    def __and__(self, other):
        """Реализует оператор &, который определяет, пересекаются ли 
        здания self и other первым способом (через углы)"""
        # для вычисления b1 & b2 питон вызывает метод b1.__and__(b2) 
        # здесь удобно использовать метод corners и перегруженный оператор in
        b1, b2 = self, other    # удобно переобозвать их - так короче и симметричнее
        
        left = max(b1.x1, b2.x1) # левая
        bottom = max(b1.y1,b2.y1) # нижняя
        right = min(b1.x2, b2.x2) # правая
        top = min(b1.y2, b2.y2) # верхняя
        
        width = right - left # ширина пересечения
        height = top - bottom # высота пересечения

        # если ширина и высота области пересечения меньше 0, то не пересекаются
        if width < 0 or height < 0:
            return False

        return True
        
    

Здесь pass - это аналог пустых скобок в си, чтобы питон понял, что это не ошибка в отступах,  
а действительно пустое тело функции/класса/цикла и т.д., например, бесконечный цикл  
записывается так:

    while True:  
        pass


In [12]:
def build_random(c):
    """Эта функция генерирует экземпляр здания типа с
    ненулевой площади со случайными координатами (для тестов).
    Например, build_random(Building1) "строит" здание типа Building1."""
    x, y, dx, dy = [randint(1, 20) for k in range(4)]
    return c(x, y, x+dx, y+dy)

Этот тест ищет такие пары зданий, для которых эти два метода поиска пересечений дают разные  
результаты. Он должен автоматически найти ситуацию с рис.2с из задания:

In [13]:
from time import sleep
r1, r2 = None, None
for i in range(1000):
    b1, b2 = build_random(Building1), build_random(Building2)
    if r1 is None:
        r1, r2 = vis(b1, b2)
    else:
        r1.data_source.data.update(top=[b1.y2], bottom=[b1.y1], left=[b1.x1], right=[b1.x2])
        r2.data_source.data.update(top=[b2.y2], bottom=[b2.y1], left=[b2.x1], right=[b2.x2])
        push_notebook()
    if b1 & b2 != b2 & b1:
        print(f'Различие найдено: b1={b1} b2={b2}')
        break
    sleep(0.1)
else:
    print('Различий не найдено')

Различие найдено: b1=Building1(12, 13, 23, 16) b2=Building1(14, 7, 16, 24)
